In [53]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import boxcox, yeojohnson
import statsmodels.regression.linear_model as srl
import statsmodels.formula.api as smf

In [54]:
def load_data():
        '''
        This function loads the data from a csv file.
        
        Returns:
            df (DataFrame): The loaded data.
        '''
        df = pd.read_csv('failure_data.csv')
        
        df['Type'] = df['Type'].astype('category')
        without_nulls = df.dropna(subset=('Air temperature [K]', 'Process temperature [K]'))
        process_temp_gradient, process_temp_intercept = np.polyfit(without_nulls['Air temperature [K]'], without_nulls['Process temperature [K]'], deg=1)
        air_temp_gradient, air_temp_intercept = np.polyfit(without_nulls['Process temperature [K]'], without_nulls['Air temperature [K]'], deg=1)
        process_temp_nulls = df[df['Air temperature [K]'].notnull() & df['Process temperature [K]'].isnull()]
        process_temp_nulls['Process temperature [K]'] = np.round(process_temp_gradient * process_temp_nulls['Air temperature [K]'] + process_temp_intercept, 1)
        air_temp_nulls = df[df['Air temperature [K]'].isnull() & df['Process temperature [K]'].notnull()]
        air_temp_nulls['Air temperature [K]'] = np.round(air_temp_gradient * air_temp_nulls['Process temperature [K]'] + air_temp_intercept, 1)
        df.update(air_temp_nulls)
        df.update(process_temp_nulls)
        df = df.dropna(subset=('Air temperature [K]', 'Process temperature [K]'))
        df['Tool wear [min]'] = df['Tool wear [min]'].fillna(df['Tool wear [min]'].mean())

        df['Rotational speed [rpm]'] = boxcox(df['Rotational speed [rpm]'])[0]
        df['Machine failure'] = yeojohnson(df['Machine failure'] + 0.1)[0]
        df['TWF'] = yeojohnson(df['TWF'] + 0.1)[0]
        df['HDF'] = yeojohnson(df['HDF'] + 0.1)[0]
        df['PWF'] = yeojohnson(df['PWF'] + 0.1)[0]
        df['OSF'] = yeojohnson(df['OSF'] + 0.1)[0]
        df['RNF'] = yeojohnson(df['RNF'] + 0.1)[0]
        print(process_temp_gradient)
        print(air_temp_gradient)
        return df

df = load_data()


0.6504895742994061
1.1764186050600978


C:\Users\srayn\AppData\Local\Temp\ipykernel_15936\3555969118.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  process_temp_nulls['Process temperature [K]'] = np.round(process_temp_gradient * process_temp_nulls['Air temperature [K]'] + process_temp_intercept, 1)
C:\Users\srayn\AppData\Local\Temp\ipykernel_15936\3555969118.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  air_temp_nulls['Air temperature [K]'] = np.round(air_temp_gradient * air_temp_nulls['Process temperature [K]'] + air_temp_intercept,

In [5]:
import plotly.express as px
px.imshow(df.corr(), title="Correlation heatmap of dataframe")

C:\Users\srayn\AppData\Local\Temp\ipykernel_17984\390849661.py:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  px.imshow(df.corr(), title="Correlation heatmap of dataframe")


In [90]:
#model0 = srl.OLS('Air_temperature_[K]', df[['Process_temperature_[K]', 'Tool wear [min]']]).fit()
#df.columns = df.columns.str.replace(' ', '_')
#df.columns = df.columns.str.replace('[', '_')
#df.columns = df.columns.str.replace(']', '')
#df.columns
model0 = smf.ols('df[df.columns[3]] ~ df[df.columns[4]]', df).fit()
model0 = smf.ols('Rotational_speed__rpm ~ Torque__Nm', df).fit()
model0.summary()



<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                             
=================================================================================
Dep. Variable:     Rotational_speed__rpm   R-squared:                       0.843
Model:                               OLS   Adj. R-squared:                  0.843
Method:                    Least Squares   F-statistic:                 5.315e+04
Date:                   Fri, 17 Nov 2023   Prob (F-statistic):               0.00
Time:                           13:55:04   Log-Likelihood:             2.4806e+05
No. Observations:                   9921   AIC:                        -4.961e+05
Df Residuals:                       9919   BIC:                        -4.961e+05
Df Model:                              1                                         
Covariance Type:               nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.3165   1.39e-13   2.28e+12      0.000       0.317       0.317
Torque__Nm -7.774e-13   3.37e-15   -230.542      0.000   -7.84e-13   -7.71e-13
==============================================================================
Omnibus:                      473.083   Durbin-Watson:                   2.012
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              972.915
Skew:                          -0.335   Prob(JB):                    5.42e-212
Kurtosis:                       4.380   Cond. No.                         170.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""